[Hands-On Computer Vision with TensorFlow 2 -- Chapter 6: AutoEncoders](https://github.com/PacktPublishing/Hands-On-Computer-Vision-with-TensorFlow-2/tree/master/Chapter06)

**Resources**:

- [Autoencoder | Wikipedia](https://en.wikipedia.org/wiki/Autoencoder)

- [Implementing an Autoencoder in TensorFlow 2.0 | Towards Data Science](https://towardsdatascience.com/implementing-an-autoencoder-in-tensorflow-2-0-5e86126e9f7)

- [Introduction to Autoencoders | Jeremy Jordan](https://www.jeremyjordan.me/autoencoders/)